In [ ]:
# check this 
# https://zindi.africa/competitions/bloodsai-blood-spectroscopy-classification-challenge/discussions/9760


# https://github.com/DrCod/Blood-Spectroscopy-Classification-using-Deep-Learning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_dir = "/content/drive/MyDrive/zindi bloodsai"

# Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import os
import re
from lightgbm import LGBMRegressor
!pip install catboost
from catboost import CatBoostClassifier
import xgboost

# Read original datasets

In [ ]:
utrain = pd.read_csv(f"{root_dir}/Update_train.csv")
utest = pd.read_csv(f"{root_dir}/Updated_Test.csv")

In [ ]:
utrain['std'].describe()

count    29160.000000
mean         0.007525
std          0.004625
min          0.002400
25%          0.004500
50%          0.006100
75%          0.008900
max          0.028700
Name: std, dtype: float64

In [ ]:
utrain.shape

(29160, 178)

In [ ]:
utrain[utrain['std'] <= 0.005]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,donation_id
0,0.520883,0.528200,0.529795,0.537382,0.540265,0.546260,0.550882,0.558865,0.571352,0.585775,0.600688,0.612949,0.617855,0.621710,0.622049,0.623373,0.620610,0.619479,0.616463,0.612414,0.609885,0.605408,0.602115,0.597710,0.593485,0.592332,0.588680,0.587396,0.585577,0.585244,0.583597,0.584460,0.584548,0.586147,0.589067,0.591360,0.595636,0.599054,0.602856,0.607817,...,1.468505,1.454910,1.434286,1.443249,1.417403,1.418581,1.403326,1.395943,1.386288,1.381532,1.372256,1.367083,1.362999,1.353013,1.345374,1.341283,1.323804,1.331071,1.332295,1.328928,1.309800,1.328064,1.329464,1.314664,1.313583,1.313329,1.312243,1.310330,1.353629,1.336609,1.319344,1.278294,0,0.0026,39.27,44.17,low,ok,high,2250
1,0.529109,0.534852,0.541075,0.544269,0.546594,0.555183,0.561367,0.567637,0.582427,0.595801,0.608619,0.619217,0.625162,0.627111,0.629326,0.628116,0.627815,0.623644,0.620762,0.617781,0.613032,0.610404,0.605528,0.602216,0.598417,0.595489,0.593692,0.591697,0.590110,0.589316,0.589131,0.587455,0.589060,0.589305,0.591252,0.594853,0.598545,0.601338,0.605673,0.610725,...,1.477741,1.457003,1.445143,1.439036,1.434363,1.422782,1.408831,1.405730,1.394905,1.386579,1.373887,1.376208,1.360633,1.358802,1.353009,1.344997,1.332883,1.324965,1.339512,1.336343,1.322957,1.311612,1.316437,1.316180,1.316063,1.320698,1.321068,1.286939,1.301804,1.314457,1.282723,1.267465,1,0.0026,39.39,44.09,low,ok,high,2250
2,0.528434,0.532036,0.540413,0.541964,0.549347,0.555902,0.561458,0.567824,0.579568,0.593048,0.607972,0.617558,0.624398,0.626222,0.627771,0.626445,0.623953,0.622616,0.618701,0.615181,0.611624,0.608587,0.605730,0.601288,0.597154,0.595654,0.592535,0.591893,0.589885,0.588528,0.587108,0.586997,0.589043,0.588888,0.591670,0.594962,0.597498,0.600767,0.604963,0.612166,...,1.468838,1.456374,1.450920,1.438182,1.434188,1.427201,1.413866,1.403336,1.404049,1.383698,1.378195,1.374761,1.373550,1.362671,1.350157,1.344038,1.341004,1.335655,1.333660,1.325491,1.335868,1.334114,1.327512,1.339627,1.322394,1.328299,1.311948,1.310330,1.311108,1.283048,1.287730,1.266464,2,0.0026,39.50,44.00,low,ok,high,2250
3,0.530528,0.531880,0.539558,0.542433,0.549097,0.556224,0.561398,0.566963,0.577525,0.591982,0.607109,0.616252,0.622437,0.625715,0.626799,0.625676,0.625084,0.622652,0.618510,0.614896,0.611060,0.608285,0.603976,0.600319,0.596856,0.594169,0.591334,0.589620,0.589549,0.587194,0.586685,0.587205,0.587368,0.588510,0.592029,0.593472,0.597904,0.601190,0.604864,0.610652,...,1.474065,1.456521,1.456408,1.445327,1.429233,1.422914,1.422161,1.405341,1.403060,1.390475,1.381461,1.370967,1.370843,1.358100,1.362090,1.352020,1.340746,1.345123,1.332574,1.345660,1.349603,1.329325,1.331542,1.347718,1.321111,1.320285,1.303363,1.306762,1.297632,1.314811,1.314408,1.342224,3,0.0026,39.60,43.89,low,ok,high,2250
4,0.527530,0.536424,0.538207,0.541504,0.550546,0.555203,0.560394,0.567352,0.579438,0.592049,0.606031,0.617109,0.624688,0.626810,0.628163,0.627580,0.625115,0.623415

In [ ]:
os.listdir(root_dir)

['Exports_ldl_rows_135_full_trimmed.csv',
 'Exports_ldl_rows_135_full.csv',
 'Exports_hdl_rows_162_full_trimmed.csv',
 'Exports_hdl_rows_162_full.csv',
 'Export_hgb_rows_965_full_trimmed.csv',
 'Export_hgb_rows_965_full.csv',
 'Exports_hdl_rows_483_full.csv',
 'Exports_hdl_rows_483_full_trimmed.csv',
 'Exports_ldl_rows_410_full.csv',
 'Exports_ldl_rows_410_full_trimmed.csv',
 'Exports_hgb_rows_2457_full_trimmed.csv',
 'Exports_hgb_rows_2457_full.csv',
 'Test_trimmed.csv',
 'Test.csv',
 'Train_trimmed.csv',
 'Train.csv',
 'Updated_Test_trimmed.csv',
 'Updated_Sample_Submission.csv',
 'Updated_Test.csv',
 'US_PATENT_DETAILS.pdf',
 'notebooks',
 'Update_train.csv']

# Using extra files to predict HGB, cholesteral and HDL value

In [ ]:
filenames = ['Exports_ldl_rows_135_full.csv','Exports_hdl_rows_162_full.csv','Export_hgb_rows_965_full.csv']
extra_df = []
for e,filename in enumerate(filenames):
  extra_df.append(pd.read_csv(f"{root_dir}/{filename}"))

ldl = extra_df[0].copy()
hdl = extra_df[1].copy()
hgb = extra_df[2].copy()

In [ ]:
ldl['absorbance'] = extra_df[0]['absorbance'].str.split(" ")
hdl['absorbance'] = extra_df[1]['absorbance'].str.split(" ")
hgb['absorbance'] = extra_df[2]['absorbance'].str.split(" ")

In [ ]:
ldl.reset_index(inplace=True)
hdl.reset_index(inplace=True)
hgb.reset_index(inplace=True)

In [ ]:
ldl.head()

,index,donation_id,scan_number,absorbance,std,temperature,humidity,cholesterol_ldl_human,cholesterol_ldl_value
0,0,10908,1,"[0.55510412634195, 0.55788763187541, 0.5608186...",0.0068,36.46,52.76,low,31.9
1,1,10908,2,"[0.55576159285836, 0.56053079184389, 0.5584783...",0.0068,36.58,52.64,low,31.9
2,2,10908,3,"[0.56259720137242, 0.56234323953026, 0.5683397...",0.0068,36.72,52.54,low,31.9
3,3,10908,4,"[0.56115994621281, 0.56021635422528, 0.5614387...",0.0068,36.83,52.48,low,31.9
4,4,10908,5,"[0.56261005545562, 0.56536574978599, 0.5640406...",0.0068,36.93,52.37,low,31.9


In [ ]:
tmp = ldl.absorbance.apply(pd.Series)
columns = [f'absorbance{i}' for i in range(170)]
tmp.columns = columns
ldl = pd.concat([ldl,tmp],axis=1)
ldl.drop("absorbance",axis=1,inplace=True)

tmp = hdl.absorbance.apply(pd.Series)
columns = [f'absorbance{i}' for i in range(170)]
tmp.columns = columns
hdl = pd.concat([hdl,tmp],axis=1)
hdl.drop("absorbance",axis=1,inplace=True)

tmp = hgb.absorbance.apply(pd.Series)
columns = [f'absorbance{i}' for i in range(170)]
tmp.columns = columns
hgb = pd.concat([hgb,tmp],axis=1)
hgb.drop("absorbance",axis=1,inplace=True)

In [ ]:
ldl[columns] = ldl[columns].apply(pd.to_numeric, errors='coerce', axis=1)
hdl[columns] = hdl[columns].apply(pd.to_numeric, errors='coerce', axis=1)
hgb[columns] = hgb[columns].apply(pd.to_numeric, errors='coerce', axis=1)

In [ ]:
def inter_build_model(df, target, utrain, utest):
  X = df.drop(target,axis=1)
  y = df[target]
  model = LGBMRegressor(iterations=5000, seed=88)
  model.fit(X, y)
  utrain = utrain[X.columns]
  utest = utest[X.columns]
  y_utrain = model.predict(utrain)
  y_utest = model.predict(utest)
  return y_utrain, y_utest

In [ ]:
ldl.drop(['index','donation_id','scan_number'], axis=1, inplace=True)
hdl.drop(['index','donation_id','scan_number'], axis=1, inplace=True)
hgb.drop(['index','donation_id','scan_number'], axis=1, inplace=True)

In [ ]:
hdl.head()

,std,temperature,humidity,hdl_cholesterol_human,hdl_cholesterol_value,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,...,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169
0,0.0056,42.38,45.58,low,42.2,0.522818,0.519492,0.526442,0.532444,0.535322,0.544064,0.549396,0.552642,0.558084,0.568863,0.580020,0.589029,0.594204,0.598358,0.599254,0.598260,0.598061,0.595966,0.593763,0.589441,0.587294,0.584787,0.581781,0.577368,0.575895,0.570784,0.569226,0.567678,0.566501,0.564068,0.562860,0.561627,0.561655,0.562494,0.562632,...,1.474882,1.467491,1.458080,1.446448,1.439830,1.429815,1.422551,1.408935,1.404862,1.401934,1.383773,1.370775,1.361730,1.359061,1.354568,1.342622,1.333530,1.322590,1.314266,1.313720,1.301026,1.296812,1.286587,1.284235,1.279445,1.275701,1.278848,1.270910,1.263785,1.265718,1.263782,1.264500,1.260582,1.241527,1.244724,1.220310,1.188683,1.201894,1.233872,1.208292
1,0.0056,42.48,45.47,low,42.2,0.519231,0.522352,0.522907,0.529875,0.535889,0.541123,0.546886,0.550454,0.558756,0.567580,0.578697,0.586826,0.594560,0.597302,0.599042,0.597930,0.598053,0.594468,0.591615,0.589538,0.587169,0.583541,0.580349,0.577152,0.573878,0.571097,0.568486,0.566140,0.564856,0.562663,0.562267,0.560699,0.561819,0.562188,0.563943,...,1.478037,1.467077,1.458287,1.443931,1.438890,1.428962,1.420491,1.413987,1.395603,1.399419,1.386989,1.375359,1.367559,1.359121,1.349818,1.335877,1.333012,1.322548,1.314735,1.305575,1.304774,1.295078,1.290389,1.284843,1.277745,1.277307,1.266634,1.260994,1.262008,1.264432,1.264688,1.263147,1.247994,1.243765,1.234483,1.214341,1.209225,1.181781,1.197834,1.209063
2,0.0056,42.58,45.37,low,42.2,0.517213,0.518496,0.524559,0.529592,0.536908,0.541784,0.545905,0.551181,0.558595,0.566122,0.578389,0.589054,0.593247,0.598129,0.598320,0.597657,0.597108,0.594709,0.590733,0.588614,0.585974,0.583150,0.580283,0.575103,0.574530,0.570946,0.567081,0.566705,0.564097,0.561919,0.561875,0.560474,0.561116,0.562146,0.562532,...,1.472573,1.462473,1.456203,1.446336,1.437765,1.425136,1.421310,1.411128,1.404630,1.391498,1.384843,1.368908,1.365020,1.360490,1.350670,1.335675,1.328751,1.317967,1.305701,1.313544,1.311138,1.290662,1.289921,1.289099,1.270472,1.272557,1.265078,1.266144,1.260752,1.259777,1.262554,1.258289,1.251042,1.247328,1.246725,1.196161,1.198100,1.210783,1.217032,1.179100
3,0.0056,42.68,45.32,low,42.2,0.520227,0.520379,0.526975,0.532377,0.536789,0.543567,0.546632,0.550731,0.559143,0.568397,0.578436,0.589762,0.594959,0.599709,0.599754,0.599187,0.597208,0.595744,0.592680,0.590016,0.586687,0.584683,0.580813,0.577317,0.574258,0.570839,0.567650,0.566792,0.564356,0.562807,0.562584,0.562271,0.562589,0.562352,0.563429,...,1.482247,1.463741,1.459492,1.446766,1.443686,1.430897,1.419218,1.414719,1.401722,1.395770,1.383269,1.367862,1.367062,1.359061,1.350345,1.341105,1.329838,1.326985,1.321873,1.303073,1.305940,1.300001,1.281186,1.274333,1.269271,1.275520,1.266106,1.262659,1.264475,1.257874,1.253993,1.258872,1.261109,1.241730,1.245152,1.222925,1.194124,1.184085,1.202892,1.190049
4,0.0056,42.75,45.21,low,42.2,0.517087,0.521615,0.526704,0.527695,0.534110,0.542949,0

In [ ]:
hgb.head()

,std,temperature,humidity,hemoglobin(hgb)_human,hemoglobin(hgb)_value,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,...,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169
0,0.013,46.59,35.64,low,11.1,0.498957,0.502001,0.505158,0.510112,0.519614,0.525038,0.531707,0.536456,0.544490,0.554882,0.568864,0.578752,0.585117,0.587167,0.589699,0.588177,0.587462,0.584019,0.581313,0.576205,0.572518,0.569396,0.564985,0.563227,0.558967,0.555485,0.552817,0.551409,0.549249,0.547718,0.548274,0.548499,0.547561,0.549274,0.551982,...,1.621463,1.610318,1.598105,1.584694,1.564622,1.550254,1.537327,1.528975,1.515458,1.501863,1.486598,1.473366,1.460086,1.448081,1.434463,1.425135,1.404409,1.396854,1.391813,1.380213,1.378918,1.369317,1.365136,1.347425,1.343505,1.333850,1.329628,1.319852,1.319314,1.320304,1.319681,1.318485,1.309708,1.307083,1.290866,1.265206,1.251281,1.210329,1.223653,1.243030
1,0.013,46.64,35.69,low,11.1,0.500375,0.501432,0.503892,0.509615,0.519364,0.526547,0.532646,0.538023,0.543226,0.554655,0.569531,0.579040,0.585814,0.587929,0.589054,0.588185,0.587796,0.583725,0.581149,0.576404,0.573004,0.570519,0.566236,0.562267,0.559979,0.555189,0.552179,0.551818,0.550124,0.548238,0.548790,0.548107,0.549319,0.550215,0.552171,...,1.635944,1.626366,1.610258,1.581206,1.578038,1.553045,1.542974,1.528373,1.516258,1.503581,1.492374,1.472773,1.462683,1.451926,1.441053,1.430414,1.411297,1.402478,1.397747,1.380651,1.376955,1.369235,1.359629,1.353319,1.344261,1.332314,1.339138,1.326698,1.329835,1.323160,1.326734,1.323951,1.326042,1.309541,1.281151,1.268786,1.241929,1.245091,1.282063,1.250729
2,0.013,46.67,35.79,low,11.1,0.500164,0.499449,0.503483,0.512658,0.519165,0.523212,0.531598,0.537450,0.545624,0.556236,0.568931,0.579766,0.585296,0.588940,0.590330,0.590887,0.587176,0.584786,0.581183,0.577195,0.572496,0.570707,0.565551,0.562590,0.559894,0.554655,0.553944,0.551915,0.550255,0.549119,0.549057,0.549088,0.548753,0.550469,0.552913,...,1.640200,1.623898,1.610793,1.595937,1.578816,1.560780,1.546321,1.536037,1.526901,1.502485,1.494475,1.468770,1.467145,1.460455,1.450271,1.431775,1.421509,1.416681,1.399457,1.395260,1.380277,1.378133,1.362388,1.355378,1.347298,1.344546,1.335749,1.338440,1.336598,1.323965,1.328600,1.321662,1.319562,1.311452,1.306259,1.274057,1.260544,1.244386,1.276481,1.244103
3,0.013,46.70,35.87,low,11.1,0.499012,0.500796,0.502940,0.510793,0.517116,0.526297,0.531559,0.538274,0.544656,0.554955,0.568889,0.579968,0.585528,0.589757,0.589500,0.591089,0.588290,0.584050,0.580339,0.577623,0.573361,0.570689,0.566400,0.563390,0.559451,0.555661,0.552680,0.552240,0.550035,0.548813,0.548815,0.549050,0.549582,0.550692,0.552628,...,1.647657,1.636828,1.627185,1.598641,1.587942,1.564259,1.556053,1.536727,1.531906,1.514902,1.503444,1.485796,1.475545,1.463156,1.448866,1.436267,1.425129,1.414519,1.406630,1.387632,1.387947,1.383544,1.372665,1.357504,1.356838,1.355171,1.347045,1.340624,1.343364,1.330207,1.331451,1.326293,1.315661,1.321996,1.310705,1.279207,1.260471,1.269043,1.268761,1.294267
4,0.013,46.75,35.91,low,11.1,0.498714,0.499235,0.503830,0.507867,0.515657,0.525013,0.5299

In [ ]:
ldl.drop("cholesterol_ldl_human",axis=1,inplace=True)

In [ ]:
hgb.drop("hemoglobin(hgb)_human",axis=1,inplace=True)
hdl.drop("hdl_cholesterol_human",axis=1,inplace=True)

In [ ]:
utrain_main = utrain.copy()
utest_main = utest.copy()
utrain['cholesterol_ldl_value'], utest['cholesterol_ldl_value'] = inter_build_model(ldl, "cholesterol_ldl_value", utrain_main, utest_main)
utrain['hemoglobin(hgb)_value'], utest['hemoglobin(hgb)_value'] = inter_build_model(hgb, "hemoglobin(hgb)_value", utrain_main, utest_main)
utrain['hdl_cholesterol_value'], utest['hdl_cholesterol_value'] = inter_build_model(hdl, "hdl_cholesterol_value", utrain_main, utest_main)

In [ ]:
target_cols = ['hdl_cholesterol_human', 'hemoglobin(hgb)_human', 'cholesterol_ldl_human']

# Basic Idea

## remove donation ID with negative values

In [ ]:
utrain = utrain[utrain['donation_id'] != 6824]  # from zindi discussion removing the value

In [ ]:
group_cols = target_cols
group_cols.extend(['donation_id'])
agg_cols = [cols for cols in list(utrain.columns) if cols not in group_cols]
utrain = utrain.groupby(group_cols, as_index=False)[agg_cols].mean()

## Compute Chi-squared value between target values to know their dependency

In [ ]:
for target in target_cols:
  for target1 in target_cols:
    if(target != target1):
      CrosstabResult=pd.crosstab(index=utrain[target],columns=utrain[target1])
      
      # importing the required function
      from scipy.stats import chi2_contingency
      
      # Performing Chi-sq test
      ChiSqResult = chi2_contingency(CrosstabResult)
      
      # P-Value is the Probability of H0 being True
      # If P-Value&gt;0.05 then only we Accept the assumption(H0)
      
      print(f'The P-Value of the ChiSq Test is: {ChiSqResult[1]} for columns {target} and {target1}')

The P-Value of the ChiSq Test is: 0.11133455909239975 for columns hdl_cholesterol_human and hemoglobin(hgb)_human
The P-Value of the ChiSq Test is: 0.00028879255476418035 for columns hdl_cholesterol_human and cholesterol_ldl_human
The P-Value of the ChiSq Test is: 0.47584966237251675 for columns hdl_cholesterol_human and donation_id
The P-Value of the ChiSq Test is: 0.11133455909239975 for columns hemoglobin(hgb)_human and hdl_cholesterol_human
The P-Value of the ChiSq Test is: 0.6041514422045116 for columns hemoglobin(hgb)_human and cholesterol_ldl_human
The P-Value of the ChiSq Test is: 0.4758496623725178 for columns hemoglobin(hgb)_human and donation_id
The P-Value of the ChiSq Test is: 0.00028879255476418035 for columns cholesterol_ldl_human and hdl_cholesterol_human
The P-Value of the ChiSq Test is: 0.6041514422045116 for columns cholesterol_ldl_human and hemoglobin(hgb)_human
The P-Value of the ChiSq Test is: 0.47584966237251675 for columns cholesterol_ldl_human and donation_id
T

From the value above cholesteral_ldl and hdl_cholesteral has high significance.
They are highly correlated. Hence while predicting we can use one of this target variable (predicted before) as independent variable to predict the other.

In [ ]:
display(utrain.isnull().sum().to_frame().rename({0:"col"},axis=1)['col'].describe())

count    181.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: col, dtype: float64

In [ ]:
group_cols = ['donation_id']
group_cols.extend(target_cols)

In [ ]:
group_cols

['donation_id',
 'hdl_cholesterol_human',
 'hemoglobin(hgb)_human',
 'cholesterol_ldl_human',
 'donation_id']

In [ ]:
utrain.shape

(485, 181)

# Modelling

In [ ]:
utrain.drop(['id','donation_id'], axis=1,inplace=True)
utest.drop(['id','donation_id'], axis=1,inplace=True)

In [ ]:
train = utrain.copy()
test = utest.copy()

## Standardization

In [ ]:
ms = MinMaxScaler()
ms.fit(utrain.iloc[:,3:])
utrain.iloc[:,3:] = ms.transform(utrain.iloc[:,3:])
utest.iloc[:,1:] = ms.transform(utest.iloc[:,1:])

In [ ]:
train.head()

,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,...,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,std,temperature,humidity,cholesterol_ldl_value,hemoglobin(hgb)_value,hdl_cholesterol_value
0,high,high,high,0.435382,0.438791,0.439986,0.445657,0.452341,0.456131,0.461042,0.464565,0.470644,0.481887,0.491988,0.502120,0.508027,0.510486,0.512243,0.512177,0.510819,0.509138,0.506801,0.504390,0.502583,0.500152,0.497625,0.492438,0.489046,0.485886,0.483728,0.481719,0.479639,0.479588,0.478937,0.478346,0.478900,0.480725,0.482721,0.485003,0.487446,...,1.314290,1.303257,1.292808,1.279820,1.269062,1.258161,1.248304,1.238701,1.229192,1.218939,1.210518,1.203541,1.195683,1.185874,1.178724,1.173099,1.166186,1.159013,1.153107,1.148547,1.144742,1.140711,1.139046,1.136737,1.136627,1.136458,1.136709,1.132411,1.124516,1.115935,1.106472,1.097726,1.109001,1.100418,0.0113,37.998000,47.734833,132.178430,14.940907,52.812994
1,high,high,high,0.519160,0.520758,0.524135,0.531012,0.538598,0.545950,0.551802,0.555630,0.561578,0.571122,0.583781,0.597792,0.604226,0.606426,0.605623,0.603955,0.601594,0.598685,0.595824,0.592542,0.590142,0.587006,0.583625,0.579231,0.576540,0.573631,0.570963,0.568935,0.567159,0.565709,0.564750,0.564495,0.564729,0.565203,0.566842,0.569227,0.571741,...,1.468765,1.456055,1.444760,1.433836,1.422941,1.412292,1.401684,1.388897,1.382493,1.368209,1.358968,1.351246,1.343646,1.333320,1.326867,1.317985,1.311786,1.305259,1.297134,1.290629,1.285784,1.281875,1.278125,1.276019,1.275182,1.271185,1.267642,1.260291,1.246926,1.228493,1.199567,1.178169,1.169879,1.144502,0.0147,47.230167,40.741667,111.388203,13.673791,50.122464
2,high,high,ok,0.494756,0.497914,0.502072,0.507431,0.515865,0.526259,0.534075,0.538439,0.543844,0.552834,0.565687,0.577726,0.583598,0.585878,0.587398,0.587713,0.586193,0.583865,0.580264,0.577187,0.573481,0.570211,0.566501,0.562914,0.559336,0.556059,0.553802,0.551824,0.550055,0.548326,0.547856,0.546964,0.546745,0.547541,0.548595,0.550396,0.552749,...,1.513356,1.499190,1.484499,1.469222,1.454777,1.439083,1.426316,1.412365,1.403143,1.389031,1.376516,1.368608,1.356369,1.346306,1.339013,1.330147,1.323601,1.315271,1.306949,1.301145,1.296765,1.294005,1.289627,1.288122,1.286766,1.283650,1.280417,1.269769,1.254666,1.236129,1.225069,1.216926,1.232705,1.220495,0.0040,39.546167,22.925667,107.359531,13.294403,48.538761
3,high,high,ok,0.522001,0.522379,0.525600,0.528251,0.534360,0.539577,0.547697,0.553577,0.561679,0.573297,0.585709,0.596667,0.604360,0.607021,0.607954,0.607658,0.606387,0.604386,0.600596,0.598178,0.595337,0.592203,0.588721,0.584361,0.581209,0.577746,0.574427,0.572743,0.570773,0.568801,0.567888,0.567475,0.567940,0.568739,0.570565,0.573429,0.576382,...,1.405369,1.397321,1.389552,1.381768,1.373328,1.365318,1.355736,1.348275,1.342555,1.333050,1.325216,1.319229,1.312549,1.306034,1.300922,1.294852,1.288314,1.282949,1.279211,1.274849,1.271966,1.269541,1.268122,1.267387,1.266062,1.264804,1.265694,1.260715,1.256271,1.253121,1.254148,1.254326,1.269868,1.258666,0.0249,37.359167,25.772833,99.440643,15.103285,54.159565
4,high,low,high,0.553549

## Target split ups

In [ ]:
for col in ['hdl_cholesterol_human','hemoglobin(hgb)_human','cholesterol_ldl_human']:
  display(utrain[col].value_counts())

ok      249
low     131
high    105
Name: hdl_cholesterol_human, dtype: int64

ok      413
high     42
low      30
Name: hemoglobin(hgb)_human, dtype: int64

ok      259
high    204
low      22
Name: cholesterol_ldl_human, dtype: int64

In [ ]:
train = utrain.iloc[:, 3:]

In [ ]:
target_cols = list(utrain.columns[:3])

In [ ]:
target_cols

['hdl_cholesterol_human', 'hemoglobin(hgb)_human', 'cholesterol_ldl_human']

In [ ]:
mapping = {"low":0,"ok":1,"high":2}
for target in target_cols:
  utrain[target] = utrain[target].map(mapping)

In [ ]:
test = utest.iloc[:,1:].copy()

In [ ]:
train.shape

(485, 176)

In [ ]:
test = test[train.columns]

In [ ]:
train.drop("std",axis=1,inplace=True)
test.drop("std",axis=1,inplace=True)

## LGBM , Catboost and XGBoost

In [ ]:
def build_model(target, test, utest, model_name = "lgbm"):
  X = train.copy()
  y = utrain[target]
  if(target == "cholesterol_ldl_human"):
    train_cols = list(X.columns)
    test_cols = list(X.columns)
    train_cols.extend(["hdl_cholesterol_human"])
    X = utrain[train_cols].copy()
    test_cols.extend([f"hdl_cholesterol_human_{model_name}"])
    test = utest[test_cols]
    test.rename({f"hdl_cholesterol_human_{model_name}":f"hdl_cholesterol_human"},axis=1,inplace=True)
  X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, stratify=y, random_state=88)
  if(model_name=="lgbm"):
    model = LGBMClassifier(iterations=5000, seed=88)
  elif(model_name=="catboost"):
    model = CatBoostClassifier(iterations=5000, random_seed=88)
  else:
    model = xgboost.XGBClassifier(iterations=5000, seed=88)
 
  model.fit(X_train, y_train)
  y_val_pred = model.predict(X_val)
  y_test = model.predict(test)
  return y_test

In [ ]:
utest_v1 = utest.copy()
model_names = ['lgbm','catboost','xgb']
for model in model_names:
  for target in target_cols:
    preds = build_model(target, test, utest, model)
    utest[target+"_"+model] = preds

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Streaming output truncated to the last 5000 lines.
0:	learn: 1.0797250	total: 159ms	remaining: 13m 14s
1:	learn: 1.0616281	total: 272ms	remaining: 11m 20s
2:	learn: 1.0455194	total: 378ms	remaining: 10m 29s
3:	learn: 1.0286622	total: 505ms	remaining: 10m 30s
4:	learn: 1.0125995	total: 611ms	remaining: 10m 10s
5:	learn: 0.9965629	total: 724ms	remaining: 10m 2s
6:	learn: 0.9818884	total: 839ms	remaining: 9m 58s
7:	learn: 0.9684805	total: 954ms	remaining: 9m 55s
8:	learn: 0.9549576	total: 1.06s	remaining: 9m 49s
9:	learn: 0.9423351	total: 1.19s	remaining: 9m 54s
10:	learn: 0.9300473	total: 1.3s	remaining: 9m 50s
11:	learn: 0.9172656	total: 1.43s	remaining: 9m 54s
12:	learn: 0.9048054	total: 1.54s	remaining: 9m 50s
13:	learn: 0.8934071	total: 1.66s	remaining: 9m 49s
14:	learn: 0.8820381	total: 1.78s	remaining: 9m 50s
15:	learn: 0.8714804	total: 1.89s	remaining: 9m 49s
16:	learn: 0.8605044	total: 2s	remaining: 9m 46s
17:	learn: 0.8504902	total: 2.12s	remaining: 9m 46s
18:	learn: 0.8396273	t

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Streaming output truncated to the last 5000 lines.
0:	learn: 1.0914541	total: 155ms	remaining: 12m 55s
1:	learn: 1.0852966	total: 281ms	remaining: 11m 41s
2:	learn: 1.0784363	total: 392ms	remaining: 10m 52s
3:	learn: 1.0715862	total: 509ms	remaining: 10m 35s
4:	learn: 1.0654108	total: 620ms	remaining: 10m 19s
5:	learn: 1.0595708	total: 734ms	remaining: 10m 11s
6:	learn: 1.0537277	total: 842ms	remaining: 10m
7:	learn: 1.0482205	total: 967ms	remaining: 10m 3s
8:	learn: 1.0422136	total: 1.08s	remaining: 9m 59s
9:	learn: 1.0369130	total: 1.19s	remaining: 9m 54s
10:	learn: 1.0312224	total: 1.3s	remaining: 9m 50s
11:	learn: 1.0258097	total: 1.41s	remaining: 9m 47s
12:	learn: 1.0209119	total: 1.52s	remaining: 9m 45s
13:	learn: 1.0163100	total: 1.64s	remaining: 9m 45s
14:	learn: 1.0111829	total: 1.75s	remaining: 9m 41s
15:	learn: 1.0061973	total: 1.86s	remaining: 9m 40s
16:	learn: 1.0009356	total: 1.99s	remaining: 9m 43s
17:	learn: 0.9962950	total: 2.1s	remaining: 9m 42s
18:	learn: 0.9919205	t

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Format to submission dataset

In [ ]:
sample_submission = pd.read_csv(root_dir + "/Updated_Sample_Submission.csv")

In [ ]:
test_old = pd.read_csv(root_dir + "/Test.csv")

In [ ]:
test_old.shape

(3660, 173)

In [ ]:
test_main = pd.read_csv(root_dir + "/Updated_Test.csv")

In [ ]:
utest_v1.shape

(3660, 177)

In [ ]:
for model in model_names:
  for target in target_cols:
    display(utest[target+"_"+model].value_counts())

1    2805
0     477
2     378
Name: hdl_cholesterol_human_lgbm, dtype: int64

1    3636
0      19
2       5
Name: hemoglobin(hgb)_human_lgbm, dtype: int64

1    2341
2    1317
0       2
Name: cholesterol_ldl_human_lgbm, dtype: int64

1    2859
0     473
2     328
Name: hdl_cholesterol_human_catboost, dtype: int64

1    3652
0       6
2       2
Name: hemoglobin(hgb)_human_catboost, dtype: int64

1    2353
2    1297
0      10
Name: cholesterol_ldl_human_catboost, dtype: int64

1    2903
0     444
2     313
Name: hdl_cholesterol_human_xgb, dtype: int64

1    3647
0       8
2       5
Name: hemoglobin(hgb)_human_xgb, dtype: int64

1    2509
2    1151
Name: cholesterol_ldl_human_xgb, dtype: int64

In [ ]:
mapping_inv = dict((v, k) for k, v in mapping.items())

In [ ]:
for model in model_names:
  for target in target_cols:
    utest[target+"_"+model] = utest[target+"_"+model].map(mapping_inv)

In [ ]:
utest.head()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,std,temperature,humidity,cholesterol_ldl_value,hemoglobin(hgb)_value,hdl_cholesterol_value,hdl_cholesterol_human_lgbm,hemoglobin(hgb)_human_lgbm,cholesterol_ldl_human_lgbm,hdl_cholesterol_human_catboost,hemoglobin(hgb)_human_catboost,cholesterol_ldl_human_catboost,hdl_cholesterol_human_xgb,hemoglobin(hgb)_human_xgb,cholesterol_ldl_human_xgb
0,ID_KWUAQLRM,0.684543,0.679512,0.675039,0.666285,0.659439,0.650148,0.628562,0.621608,0.622698,0.610835,0.610486,0.595050,0.575566,0.574383,0.567895,0.563007,0.566309,0.563482,0.557618,0.556836,0.550685,0.546675,0.540045,0.547008,0.535119,0.532353,0.529146,0.533505,0.530783,0.529356,0.524006,0.522994,0.519863,0.519188,0.517748,0.517403,0.517998,0.509536,0.510126,...,0.347309,0.335991,0.351478,0.354512,0.349200,0.347267,0.354801,0.357656,0.352955,0.353952,0.361318,0.359929,0.356663,0.363893,0.368648,0.371693,0.352544,0.386629,0.403298,0.400017,0.384417,0.422917,0.401344,0.372617,0.490682,0.011407,0.211313,0.224177,0.304147,0.280481,0.169268,ok,ok,high,ok,ok,high,ok,ok,high
1,ID_FN8SIVJJ,0.684261,0.684419,0.670504,0.674853,0.674211,0.657172,0.640618,0.628983,0.625286,0.618534,0.612962,0.598029,0.578077,0.577423,0.574318,0.568283,0.569925,0.571298,0.561953,0.554381,0.554959,0.550265,0.548060,0.544254,0.536553,0.540196,0.536941,0.534434,0.532663,0.533423,0.529110,0.526129,0.524908,0.523120,0.517757,0.517385,0.518307,0.511256,0.511701,...,0.345499,0.325608,0.334888,0.351516,0.344993,0.364179,0.354250,0.348194,0.370125,0.357683,0.346506,0.368748,0.358253,0.360816,0.368380,0.366527,0.386260,0.390193,0.408835,0.385055,0.396445,0.375846,0.405631,0.399136,0.480644,0.011407,0.216922,0.223062,0.331391,0.274698,0.165917,ok,ok,high,ok,ok,ok,ok,ok,high
2,ID_CI4TQCHI,0.683917,0.683499,0.676088,0.674047,0.672045,0.660077,0.634939,0.631345,0.626374,0.625983,0.616804,0.596763,0.585037,0.582429,0.577169,0.568491,0.572913,0.567462,0.561701,0.557577,0.556159,0.551431,0.547849,0.548644,0.538678,0.539640,0.538690,0.537543,0.533915,0.530353,0.529331,0.527998,0.526271,0.523029,0.521083,0.519900,0.517916,0.512778,0.513884,...,0.334529,0.339578,0.337139,0.349125,0.351381,0.355246,0.351451,0.357810,0.361161,0.360049,0.368603,0.384851,0.369207,0.367044,0.374834,0.382324,0.363704,0.397026,0.411431,0.422242,0.380361,0.411252,0.376760,0.394582,0.351777,0.011407,0.222020,0.220830,0.364152,0.261614,0.173005,ok,ok,high,ok,ok,ok,ok,ok,high
3,ID_URJMKNU7,0.702331,0.697812,0.702317,0.688832,0.687735,0.678089,0.662544,0.650103,0.654252,0.646886,0.632237,0.614594,0.598879,0.596608,0.593594,0.588956,0.593185,0.587468,0.584046,0.575223,0.570784,0.571329,0.566720,0.564376,0.555577,0.557518,0.554154,0.552793,0.547842,0.545507,0.543788,0.543644,0.537660,0.536379,0.533068,0.527856,0.529587,0.521888,0.519265,...,0.344661,0.349205,0.360887,0.357147,0.344804,0.357953,0.347068,0.347685,0.370604,0.352584,0.369870,0.360463,0.361994,0.383927,0.376319,0.359610,0.362424,0.385389,0.371892,0.382736,0.333062,0.386759,0.444837,0.332016,0.395368,0.011407,0.226099,0.218821,0.284616,0.291384,0.156433,ok,ok,ok,ok,ok,ok,ok,ok,ok
4,ID_HBJH67IS,0.691050,0.685254,0.675012,0.674780,0.667748,0.661080,0.643694,0.6

## Mode from 3 models predictions

In [ ]:
test_main['hdl_cholesterol_human'] = utest[['hdl_cholesterol_human_lgbm','hdl_cholesterol_human_catboost','hdl_cholesterol_human_xgb']].mode(axis=1, dropna=True)[0]
test_main['hemoglobin(hgb)_human'] = utest[['hemoglobin(hgb)_human_lgbm','hemoglobin(hgb)_human_catboost','hemoglobin(hgb)_human_xgb']].mode(axis=1, dropna=True)[0]
test_main['cholesterol_ldl_human'] = utest[['cholesterol_ldl_human_lgbm','cholesterol_ldl_human_catboost','cholesterol_ldl_human_xgb']].mode(axis=1, dropna=True)[0]

In [ ]:
final_test = test_main[['donation_id','hdl_cholesterol_human','hemoglobin(hgb)_human','cholesterol_ldl_human']]

In [ ]:
final_test['hdl_cholesterol_human'].value_counts()

In [ ]:
sample_submission.head()

In [ ]:
targets = ['hdl_cholesterol_human', 'cholesterol_ldl_human', 'hemoglobin(hgb)_human']

In [ ]:
def transform_c_hdl(row):
    return str(row["donation_id"]) + "_hdl_cholesterol_human" + "-" +  row["hdl_cholesterol_human"]
hdl_rows = pd.DataFrame(final_test[['donation_id'] + targets].apply(transform_c_hdl, axis=1))
def transform_hemo(row):
    return str(row["donation_id"]) + "_hemoglobin(hgb)_human" +  "-" + row["hemoglobin(hgb)_human"]
hemo_rows = pd.DataFrame(final_test[['donation_id'] + targets].apply(transform_hemo, axis=1))
def transform_c_ldl(row):
    return str(row["donation_id"]) + "_cholesterol_ldl_human" +  "-" + row["cholesterol_ldl_human"]
ldl_rows = pd.DataFrame(final_test[['donation_id'] + targets].apply(transform_c_ldl, axis=1))
ss = pd.concat([hdl_rows, hemo_rows, ldl_rows]).reset_index(drop=True)

In [ ]:
ss

In [ ]:
ss["target"] = ss[0].apply(lambda x: x.split("-")[1])
ss[0] = ss[0].apply(lambda x: x.split("-")[0])
ss = ss.rename(columns={0:"Donation_ID"})
ss.target.value_counts()

In [ ]:
ss

In [ ]:
ss.groupby(['Donation_ID','target']).agg(lambda x:x.value_counts().index[0])

In [ ]:
submission_183 = ss.groupby(['Donation_ID'])['target'].agg(pd.Series.mode).to_frame().reset_index()

In [ ]:
submission_183.to_csv("submission_reg_cls.csv",index=False)

In [ ]:
submission_183.shape